# TODO

https://towardsdatascience.com/winning-blackjack-using-machine-learning-681d924f197c

1. Andmestiku puhastus, ainult need asjad mida vaja on.
2. Sorteerida ära terve andmestiku peale pärast esimese kahe kaardi andmist kas käsi on soft-hand, hard-hand või paar (nagu need tabelid üleval antud lehel on). Hard-handid saame nii kätte, et kus kaks kaarti pole kas paar või üks kahest kaardist ei ole 
3. Nüüd kui meil on ära sorteeritud käed, siis iga käega peaksime välja arvutama (I assume) võiduprotsendi järgi kas me peame hittima, stay või panust duubeldama. 
4. Ja nii arvutame välja kõikide juhtudega ehk kui sul on kolm kaarti käes, neli kaarti jne.

In [3]:
import pandas as pd
import warnings

data = pd.read_csv("blkjckhands.csv", index_col=False)
data = data.drop(columns=['Unnamed: 0', 'PlayerNo', 'dealcard2', 'dealcard3', 'dealcard4', 'dealcard5', 'plybustbeat', 'dlbustbeat','plwinamt', 'dlwinamt'])


In [4]:
cond = (data.card3 == 0) & (data.card4 != 0)
data.loc[cond, ['card3', 'card4']] = data.loc[cond,['card4', 'card3']].values



#154980 - kus on kolmas kaart on õiges kohas
#190401 - kus on kolmas kaart vales kohas
#35603 - kus on kolmas ja neljas kaart vales kohas

In [5]:
cond2 = (data.card3 != 0) & (data.card4 == 0) & (data.card5 != 0)
data.loc[cond2, ['card4', 'card5']] = data.loc[cond2, ['card5', 'card4']].values

data[(data["card3"] != 0) & (data["card4"] == 0) & (data["card5"] != 0)]

,card1,card2,card3,card4,card5,sumofcards,dealcard1,sumofdeal,blkjck,winloss,ply2cardsum


In [25]:
#Sorteerisin käed vastavalt esimese kahe kaardi järgi, sealt vastavalt saame teha maatrixi siis otsustest based on win rate
#Lisaks peab jälgima seda, et meil on andmeid kus on kolmas kaart 0 aga neljas ja viies on hit. Need peame lugema pmst hittideks,
#lihtsalt 

#Ehk siis me arvutame iga käe summa järgi ning kas sellega võideti või mitte. Lisaks vaatame dealeri kaarti. Umbes kuidagi selline on
#arvutus:
# 1. Näiteks hard_hands puhul võtame summa nagu näiteks 10. 
# 2. Loeme kõik 10ned kokku. Eraldame kõik diileri kaardi juhud ning võtame näiteks kus on sinu kaartide summa on 10 ning diileri summa on 2
# 3. Loeme kokku win% kõikidel kätel kus hitidakse ning kus ei hitida. (not sure how to make this work yet).
# 4. Paneme sinu tehtud maatriksisse lõpuks 0 asemele kas me hitime selle summaga ning selle dealeri handiga või standime.

#Sarnast algoritmi saab kasutada ka paaride ning soft_handide korral ehk me võime kasutada paaride korral näiteks alguses nn paaride maatriksit,
#ning näiteks teeme statistika selle põhjal kas splittida või mitte. Sealt me edasi mõtlema ei pea, kuna edasi läheb ta nkni otsustab kas tema käsi
#on hard_hand või mis iganes. Sama ka soft_handide põhjal.

pairs = data[data["card1"] == data["card2"]].copy()
soft_hands = data[(data["card1"] == 1) | (data["card1"] == 11) | (data["card2"] == 1) | (data["card2"] == 11)].copy()
hard_hands = data[(data["card1"] != data["card2"]) & ((data["card1"] != 1) & (data["card1"] != 11) & (data["card2"] != 1) & (data["card2"] != 11))].copy()

pairs[(pairs["card1"] == 2)]
Acesubset = data[(data["card1"]==1)|(data["card1"]==11)|(data["card2"]==1)|(data["card2"]==11)]

Otsustuse dataframe on vaja panna paika siis nende kriteerimite alusel:
1. Row tähistab handi väärtust ja veerg tähistab dealeri esimest kaarti. Me peame row väärtuseks võtma ka need handid kus ntks alles viies kaart oli see mis summaks andis 20 ehk me itereerime iga rida kus ka kaartide vaheväärtus oli otsitav summa. Ühesõnaga ütleme, et me otsime hande kus kaartide summaks oli 18, st et me vaatame hande kus esimese kahe kaardi, esimese kolme kaari, esimese nelja kaardi... jne summa oli 18 ja kas siis oli HIT v oli STAND ja kas käsi lõpuks võideti või ei.

2. Me loeme HITID ja STANDID kokku ja vaatame kumma puhul suurem võidu % oli ja kanname vastavalt siis kummagi tabelisse.

In [7]:
veerud = {'2':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '3':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '4':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '5':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '6':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '7':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '8':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '9':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         'T':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         'A':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0']}
hardHands = pd.DataFrame(veerud, index=['20','19','18','17','16','15','14','13','12','11','10','9','8','7','6','5'])

In [10]:
veerud = {'2':['0','0','0','0','0','0','0','0'],
         '3':['0','0','0','0','0','0','0','0'],
         '4':['0','0','0','0','0','0','0','0'],
         '5':['0','0','0','0','0','0','0','0'],
         '6':['0','0','0','0','0','0','0','0'],
         '7':['0','0','0','0','0','0','0','0'],
         '8':['0','0','0','0','0','0','0','0'],
         '9':['0','0','0','0','0','0','0','0'],
         'T':['0','0','0','0','0','0','0','0'],
         'A':['0','0','0','0','0','0','0','0']}

softHands = pd.DataFrame(veerud, index=['A-9', 'A-8', 'A-7', 'A-6', 'A-5', 'A-4', 'A-3', 'A-2'])


veerud = {'2':['0','0','0','0','0','0','0','0','0','0'],
         '3':['0','0','0','0','0','0','0','0','0','0'],
         '4':['0','0','0','0','0','0','0','0','0','0'],
         '5':['0','0','0','0','0','0','0','0','0','0'],
         '6':['0','0','0','0','0','0','0','0','0','0'],
         '7':['0','0','0','0','0','0','0','0','0','0'],
         '8':['0','0','0','0','0','0','0','0','0','0'],
         '9':['0','0','0','0','0','0','0','0','0','0'],
         'T':['0','0','0','0','0','0','0','0','0','0'],
         'A':['0','0','0','0','0','0','0','0','0','0']}
pairTable = pd.DataFrame(veerud, index=['A-A', 'T-T', '9-9', '8-8', '7-7', '6-6', '5-5', '4-4','3-3','2-2'])

In [106]:
#Teeme ühe korra arvutuse läbi
#Võtame kaartide summaks 5 ja dealeri uphandiks 2
#temp = hard_hands[(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] + hard_hands["card5"] == 5)].copy()
def hitvstand(summa,dcard, hand):
    if(hand == 'hard'):
        summa = int(summa)
        if(dcard == "A"):
            temp = hard_hands[
                ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
                |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
                 & ((hard_hands["dealcard1"] == 1) | (hard_hands["dealcard1"] == 11))
                ].copy()
        elif(dcard == "T"):
            temp = hard_hands[
                ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
                |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
                 & (hard_hands["dealcard1"] == 10)
                ].copy()
        else:
            temp = hard_hands[
                ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
                |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
                 & (hard_hands["dealcard1"] == int(dcard))
                ].copy()


        standwin = temp[(temp["winloss"]=="Win") & ((temp["card1"] + temp["card2"]==summa) & (temp["card3"]==0))
                      |
                    ((temp["card1"]+temp["card2"]+temp["card3"]==summa) & (temp["card3"]!=0) & (temp["card4"]==0))
                     |
                     ((temp["card1"]+temp["card2"]+temp["card3"]+temp["card4"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]==0))].copy()

        hitwin = temp[(temp["winloss"]=="Win") & ((temp["card1"] + temp["card2"]==summa) & (temp["card3"]!=0))
                      |
                    ((temp["card1"]+temp["card2"]+temp["card3"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0))
                     |
                     ((temp["card1"]+temp["card2"]+temp["card3"]+temp["card4"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]!=0))].copy()
        hitwinNr = len(hitwin.index)
        standwinNr = len(standwin.index)
        
        if(hitwinNr > standwinNr):
            return 'H'
        else:
            return 'S'
    
    if(hand == 'soft'):
        teinekaart = summa.split("-")
        summa1 = 1 + int(teinekaart[1])
        summa2 = 11 + int(teinekaart[1])
        
        if(dcard == "A"):
            temp = soft_hands[
                ((soft_hands["ply2cardsum"] == summa1) | (soft_hands["ply2cardsum"] == summa2))
                & ((soft_hands["dealcard1"] == 1) | (soft_hands["dealcard1"] == 11))
            ].copy()
        elif(dcard == "T"):
            temp = soft_hands[
                ((soft_hands["ply2cardsum"] == summa1) | (soft_hands["ply2cardsum"] == summa2))
                & (soft_hands["dealcard1"] == 10)
            ].copy()
        else:
            temp = soft_hands[
                ((soft_hands["ply2cardsum"] == summa1) | (soft_hands["ply2cardsum"] == summa2))
                & (soft_hands["dealcard1"] == int(dcard))
            ].copy()
            
        hitwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                      &
                      ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                     ].copy()
        standwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                        &
                        (temp["card3"] == 0)
                       ].copy()
        hitwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                      &
                      ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                     ].copy()
        standwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                        &
                        (temp["card3"] == 0)
                       ].copy()
        hitwinNr = max(len(hitwin1.index), len(hitwin2.index))
        standwinNr = max(len(standwin1.index), len(standwin2.index))
        

        if(hitwinNr > standwinNr):
            return 'H'
        else:
            return 'S'
    
    if(hand=="pair"):
        card1 = summa[2]
        card2 = summa[0]
        if((card1=="A") & (card2=="A")):
            card11=1
            card12=11
            card21=1
            card22=11
            summa1=2
            summa2=12
            if(dcard == "A"):
                temp = pairs[
                    (((pairs["ply2cardsum"] == summa1)|(pairs["ply2cardsum"]==summa2) &
                      ((pairs["card1"]==11)&(pairs["card2"]==1))|(pairs["card1"]==1)&(pairs["card2"]==11))
                    & ((pairs["dealcard1"] == 1) | (pairs["dealcard1"] == 11)))].copy()
            elif(dcard == "T"):
                temp = pairs[
                    (((pairs["ply2cardsum"] == summa1)|(pairs["ply2cardsum"]==summa2) &
                      ((pairs["card1"]==11)&(pairs["card2"]==1))|(pairs["card1"]==1)&(pairs["card2"]==11))
                    & (pairs["dealcard1"] == 10))
                ].copy()
            else:
                temp = pairs[
                    (((pairs["ply2cardsum"] == summa1)|(pairs["ply2cardsum"]==summa2) &
                      ((pairs["card1"]==11)&(pairs["card2"]==1))|(pairs["card1"]==1)&(pairs["card2"]==11))
                    & (pairs["dealcard1"] == int(dcard)))
                ].copy()
            
            hitwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                          &
                          ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                         ].copy()
            standwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                            &
                            (temp["card3"] == 0)
                           ].copy()
            hitwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                          &
                          ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                         ].copy()
            standwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                            &
                            (temp["card3"] == 0)
                           ].copy()
            
            splitwin = Acesubset[((Acesubset["winloss"]=="Win") & (Acesubset["card1"]==card11))
                            |((Acesubset["winloss"]=="Win") & (Acesubset["card1"]==card12))
                            |((Acesubset["winloss"]=="Win") & (Acesubset["card2"]==card21))
                            |((Acesubset["winloss"]=="Win") & (Acesubset["card2"]==card22))].copy()

            hitwinprecent = '%.2f' % (max(len(hitwin1.index), len(hitwin2.index))/len(temp.index))
            standwinprecent = '%.2f' % (max(len(standwin1.index), len(standwin2.index))/len(temp.index))
            splitwinprecent = '%.2f' % ((len(splitwin.index)/len(Acesubset.index)))
            
            return str("HIT: "+str(hitwinprecent)+ " STAND:" + str(standwinprecent) +"  SPLIT: "+str(splitwinprecent)) 
            
            if (splitwinprecent >= hitwinprecent) and (splitwinprecent >= standwinprecent):
                return "P"
            elif (hitwinprecent >= splitwinprecent) and (hitwinprecent >= standwinprecent):
                return "H"
            else:
                return "S"
            
        
        else:
            if((card1=="T") & (card2=="T")):
                summa=20
                card1=10
                card2=10
            else:
                card1=int(card1)
                card2=int(card2)
                summa=card1+card2
            if(dcard == "A"):
                temp = pairs[
                    ((pairs["ply2cardsum"] == summa) 
                    & ((pairs["dealcard1"] == 1) | (pairs["dealcard1"] == 11)))].copy()
            elif(dcard == "T"):
                temp = pairs[
                    ((pairs["ply2cardsum"] == summa)
                    & (pairs["dealcard1"] == 10))
                ].copy()
            else:
                temp = pairs[
                    ((pairs["ply2cardsum"] == summa)
                    & (pairs["dealcard1"] == int(dcard)))
                ].copy()
            
            standwin = temp[(temp["winloss"]=="Win") & ((temp["card3"]==0)
                          |
                        (temp["card3"]!=0) & (temp["card4"]==0)
                         |
                         (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]==0))].copy()

            hitwin = temp[(temp["winloss"]=="Win")& ((temp["card3"]!=0)
                          |
                        (temp["card3"]!=0) & (temp["card4"]!=0)
                         |
                         (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]!=0))].copy()
            
            smaller_data=data[(data["card1"]==card1)|(data["card2"]==card2)]
            splitwin = smaller_data[((smaller_data["winloss"]=="Win") & (smaller_data["card1"]==card1))|
                                    ((smaller_data["winloss"]=="Win") & (smaller_data["card2"]==card2))]
            splitwinprecent =  '%.2f' % ((len(splitwin.index)/len(smaller_data.index)))
            hitwinprecent =  '%.2f' % (len(hitwin.index)/len(temp.index))
            standwinprecent = '%.2f' % (len(standwin.index)/len(temp.index))

            
            
            return str("HIT: "+str(hitwinprecent)+ "  STAND:" + str(standwinprecent) +"  SPLIT: "+str(splitwinprecent)) 
            
            if (splitwinprecent >= hitwinprecent) and (splitwinprecent >= standwinprecent):
                return "P"
            elif (hitwinprecent >= splitwinprecent) and (hitwinprecent >= standwinprecent):
                return "H"
            else:
                return "S"
              
    
        
        
    


In [96]:
Columns = list(hardHands.columns)
Rows = list(hardHands.index)
for i in range(len(Rows)):
    for j in range(len(Columns)):
        taht = hitvstand(Rows[i], Columns[j], 'hard')
        hardHands.iloc[i][j] = taht
        
hardHands

,2,3,4,5,6,7,8,9,T,A
20,S,S,S,S,S,S,S,S,S,S
19,S,S,S,S,S,S,S,S,S,S
18,S,S,S,S,S,S,S,S,S,S
17,S,S,S,S,S,S,S,S,S,S
16,S,S,S,S,S,H,H,H,H,H
15,S,S,S,S,S,H,H,H,H,H
14,S,S,S,S,S,H,H,H,H,H
13,S,S,S,S,S,H,H,H,H,H
12,H,H,S,S,S,H,H,H,H,H
11,H,H,H,H,H,H,H,H,H,H


In [94]:
Columns = list(softHands.columns)
Rows = list(softHands.index)
for i in range(len(Rows)):
    for j in range(len(Columns)):
        taht = hitvstand(Rows[i], Columns[j], "soft")
        softHands.iloc[i][j] = taht
softHands

,2,3,4,5,6,7,8,9,T,A
A-9,S,S,S,S,S,S,S,S,S,S
A-8,S,S,S,S,S,S,S,S,S,S
A-7,S,S,S,S,S,S,S,S,S,S
A-6,H,H,H,H,H,H,H,H,H,H
A-5,H,H,H,H,H,H,H,H,H,H
A-4,H,H,H,H,H,H,H,H,H,H
A-3,H,H,H,H,H,H,H,H,H,H
A-2,H,H,H,H,H,H,H,H,H,H


In [107]:
Columns = list(pairTable.columns)
Rows = list(pairTable.index)
for i in range(len(Rows)):
    for j in range(len(Columns)):
        taht = hitvstand(Rows[i], Columns[j], 'pair')
        pairTable.iloc[i][j] = taht
        
pairTable

,2,3,4,5,6,7,8,9,T,A
A-A,HIT: 0.00 STAND:0.00 SPLIT: 0.59,HIT: 0.00 STAND:0.00 SPLIT: 0.59,HIT: 0.40 STAND:0.00 SPLIT: 0.59,HIT: 0.40 STAND:0.00 SPLIT: 0.59,HIT: 0.41 STAND:0.00 SPLIT: 0.59,HIT: 0.00 STAND:0.00 SPLIT: 0.59,HIT: 0.00 STAND:0.00 SPLIT: 0.59,HIT: 0.00 STAND:0.00 SPLIT: 0.59,HIT: 0.00 STAND:0.00 SPLIT: 0.59,HIT: 0.00 STAND:0.00 SPLIT: 0.59
T-T,HIT: 0.00 STAND:0.77 SPLIT: 0.46,HIT: 0.00 STAND:0.77 SPLIT: 0.46,HIT: 0.00 STAND:0.78 SPLIT: 0.46,HIT: 0.00 STAND:0.79 SPLIT: 0.46,HIT: 0.00 STAND:0.81 SPLIT: 0.46,HIT: 0.00 STAND:0.84 SPLIT: 0.46,HIT: 0.00 STAND:0.87 SPLIT: 0.46,HIT: 0.00 STAND:0.81 SPLIT: 0.46,HIT: 0.00 STAND:0.55 SPLIT: 0.46,HIT: 0.00 STAND:0.51 SPLIT: 0.46
9-9,HIT: 0.00 STAND:0.48 SPLIT: 0.44,HIT: 0.00 STAND:0.53 SPLIT: 0.44,HIT: 0.00 STAND:0.53 SPLIT: 0.44,HIT: 0.00 STAND:0.50 SPLIT: 0.44,HIT: 0.00 STAND:0.58 SPLIT: 0.44,HIT: 0.00 STAND:0.58 SPLIT: 0.44,HIT: 0.00 STAND:0.37 SPLIT: 0.44,HIT: 0.00 STAND:0.41 SPLIT: 0.44,HIT: 0.00 STAND:0.32 SPLIT: 0.44,HIT: 0.00 STAND:0.28 SPLIT: 0.44
8-8,HIT: 0.00 STAND:0.40 SPLIT: 0.40,HIT: 0.00 STAND:0.39 SPLIT: 0.40,HIT: 0.00 STAND:0.36 SPLIT: 0.40,HIT: 0.00 STAND:0.45 SPLIT: 0.40,HIT: 0.00 STAND:0.40 SPLIT: 0.40,HIT: 0.27 STAND:0.27 SPLIT: 0.40,HIT: 0.24 STAND:0.24 SPLIT: 0.40,HIT: 0.24 STAND:0.24 SPLIT: 0.40,HIT: 0.20 STAND:0.20 SPLIT: 0.40,HIT: 0.16 STAND:0.16 SPLIT: 0.40
7-7,HIT: 0.00 STAND:0.37 SPLIT: 0.35,HIT: 0.00 STAND:0.33 SPLIT: 0.35,HIT: 0.00 STAND:0.40 SPLIT: 0.35,HIT: 0.00 STAND:0.37 SPLIT: 0.35,HIT: 0.00 STAND:0.45 SPLIT: 0.35,HIT: 0.29 STAND:0.29 SPLIT: 0.35,HIT: 0.31 STAND:0.31 SPLIT: 0.35,HIT: 0.26 STAND:0.26 SPLIT: 0.35,HIT: 0.21 STAND:0.21 SPLIT: 0.35,HIT: 0.17 STAND:0.17 SPLIT: 0.35
6-6,HIT: 0.39 STAND:0.39 SPLIT: 0.35,HIT: 0.35 STAND:0.35 SPLIT: 0.35,HIT: 0.00 STAND:0.39 SPLIT: 0.35,HIT: 0.00 STAND:0.43 SPLIT: 0.35,HIT: 0.00 STAND:0.48 SPLIT: 0.35,HIT: 0.36 STAND:0.36 SPLIT: 0.35,HIT: 0.35 STAND:0.35 SPLIT: 0.35,HIT: 0.29 STAND:0.29 SPLIT: 0.35,HIT: 0.25 STAND:0.25 SPLIT: 0.35,HIT: 0.24 STAND:0.24 SPLIT: 0.35
5-5,HIT: 0.54 STAND:0.54 SPLIT: 0.35,HIT: 0.60 STAND:0.60 SPLIT: 0.35,HIT: 0.57 STAND:0.57 SPLIT: 0.35,HIT: 0.59 STAND:0.59 SPLIT: 0.35,HIT: 0.57 STAND:0.57 SPLIT: 0.35,HIT: 0.63 STAND:0.61 SPLIT: 0.35,HIT: 0.60 STAND:0.58 SPLIT: 0.35,HIT: 0.48 STAND:0.47 SPLIT: 0.35,HIT: 0.40 STAND:0.39 SPLIT: 0.35,HIT: 0.35 STAND:0.34 SPLIT: 0.35
4-4,HIT: 0.47 STAND:0.47 SPLIT: 0.37,HIT: 0.48 STAND:0.47 SPLIT: 0.37,HIT: 0.51 STAND:0.51 SPLIT: 0.37,HIT: 0.53 STAND:0.53 SPLIT: 0.37,HIT: 0.52 STAND:0.51 SPLIT: 0.37,HIT: 0.52 STAND:0.49 SPLIT: 0.37,HIT: 0.37 STAND:0.33 SPLIT: 0.37,HIT: 0.33 STAND:0.30 SPLIT: 0.37,HIT: 0.30 STAND:0.28 SPLIT: 0.37,HIT: 0.24 STAND:0.21 SPLIT: 0.37
3-3,HIT: 0.39 STAND:0.36 SPLIT: 0.37,HIT: 0.44 STAND:0.40 SPLIT: 0.37,HIT: 0.43 STAND:0.40 SPLIT: 0.37,HIT: 0.48 STAND:0.45 SPLIT: 0.37,HIT: 0.46 STAND:0.44 SPLIT: 0.37,HIT: 0.35 STAND:0.29 SPLIT: 0.37,HIT: 0.34 STAND:0.29 SPLIT: 0.37,HIT: 0.28 STAND:0.22 SPLIT: 0.37,HIT: 0.24 STAND:0.19 SPLIT: 0.37,HIT: 0.19 STAND:0.16 SPLIT: 0.37
2-2,HIT: 0.39 STAND:0.34 SPLIT: 0.38,HIT: 0.41 STAND:0.34 SPLIT: 0.38,HIT: 0.46 STAND:0.42 SPLIT: 0.38,HIT: 0.49 STAND:0.45 SPLIT: 0.38,HIT: 0.44 STAND:0.41 SPLIT: 0.38,HIT: 0.40 STAND:0.28 SPLIT: 0.38,HIT: 0.34 STAND:0.24 SPLIT: 0.38,HIT: 0.31 STAND:0.22 SPLIT: 0.38,HIT: 0.28 STAND:0.18 SPLIT: 0.38,HIT: 0.21 STAND:0.14 SPLIT: 0.38


In [90]:
hitvstand("2-2", "2", 'pair')

'P'